### Neural Network Model:

In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
import pandas as pd
loan=pd.read_csv('loan.csv',low_memory=False)

In [4]:
total_num = loan.isnull().sum().sort_values(ascending=False)

perc = loan.isnull().sum()/loan.isnull().count() *100
perc1 = (round(perc,2).sort_values(ascending=False))

# Creating a data frame:
df_miss = pd.concat([total_num, perc1], axis =1 , keys =["Total Missing Values", "Percentage %"]).sort_values(by ="Percentage %", ascending = False)
top_mis = df_miss[df_miss["Percentage %"]>0]
top_mis.reset_index(inplace=True)

loan.drop(top_mis[top_mis["Percentage %"]>75]["index"], axis = 1, inplace=True)
loan.drop(["next_pymnt_d","mths_since_last_delinq"]
        , axis = 1, inplace=True)
loan.drop(["last_pymnt_d", "last_credit_pull_d", "earliest_cr_line", "issue_d", "addr_state"], axis =1 , inplace = True)
loan["tot_cur_bal"] = loan["tot_cur_bal"].fillna(loan["tot_cur_bal"].mean())

loan["tot_coll_amt"] = loan["tot_coll_amt"].fillna(0)

loan["emp_length"]=loan["emp_length"].fillna(0)

loan["revol_util"]= loan["revol_util"].fillna(loan["revol_util"].mean())

loan["collections_12_mths_ex_med"]= loan["collections_12_mths_ex_med"].fillna(loan["collections_12_mths_ex_med"].mean())

loan["acc_now_delinq"]=loan["acc_now_delinq"].fillna(loan["acc_now_delinq"].mode())

loan["delinq_2yrs"]=loan["delinq_2yrs"].fillna(0)

loan["inq_last_6mths"]= loan["inq_last_6mths"].fillna(0)

loan["pub_rec"]= loan["pub_rec"].fillna(0)

loan["acc_now_delinq"]= loan["acc_now_delinq"].fillna(0)

loan["open_acc"]= loan["open_acc"].fillna(loan["open_acc"].mean())

D:\Softwares\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [5]:
loan.dropna(axis=0,inplace=True)

In [7]:
import numpy as np
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
            "Late (16-30 days)", "Late (31-120 days)"]

# Defining a new column loan_condition to qualify good/ bad loans
loan['loan_condition'] = np.nan


def loan_condition(status):
    if status in bad_loan:
        return 'Bad Loan'
    else:
        return 'Good Loan'
    
#calling function using data frame  
loan['loan_condition'] = loan['loan_status'].apply(loan_condition)

In [8]:
loan['income_category'] = np.nan
lst = [loan]

for col in lst:
    col.loc[col['annual_inc'] <= 100000, 'income_category'] = 'Low'
    col.loc[(col['annual_inc'] > 100000) & (col['annual_inc'] <= 200000), 'income_category'] = 'Medium'
    col.loc[col['annual_inc'] > 200000, 'income_category'] = 'High' 

In [9]:
loan['loan_condition_int'] = np.nan

In [10]:
loan['loan_condition_int'] = np.nan
def to_int1(condition):
    if condition == 'Good Loan':
        return 1
    elif condition == 'Bad Loan':
        return 0

loan['loan_condition_int'] = loan['loan_condition'].apply(to_int1)

In [11]:
employment_length = ['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
                    '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a']

# Create a new column and convert emp_length to integers.

lst = [loan]
loan['emp_length_int'] = np.nan

for col in lst:
    col.loc[col['emp_length'] == '10+ years', "emp_length_int"] = 10
    col.loc[col['emp_length'] == '9 years', "emp_length_int"] = 9
    col.loc[col['emp_length'] == '8 years', "emp_length_int"] = 8
    col.loc[col['emp_length'] == '7 years', "emp_length_int"] = 7
    col.loc[col['emp_length'] == '6 years', "emp_length_int"] = 6
    col.loc[col['emp_length'] == '5 years', "emp_length_int"] = 5
    col.loc[col['emp_length'] == '4 years', "emp_length_int"] = 4
    col.loc[col['emp_length'] == '3 years', "emp_length_int"] = 3
    col.loc[col['emp_length'] == '2 years', "emp_length_int"] = 2
    col.loc[col['emp_length'] == '1 year', "emp_length_int"] = 1
    col.loc[col['emp_length'] == '< 1 year', "emp_length_int"] = 0.5
    col.loc[col['emp_length'] == 'n/a', "emp_length_int"] = 0

In [12]:
X = loan[['installment','inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_pymnt',
       'total_pymnt_inv','total_rec_int',
       'tot_coll_amt', 'tot_cur_bal',
        'loan_condition_int']].copy()
X.head()

,installment,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_pymnt,total_pymnt_inv,total_rec_int,tot_coll_amt,tot_cur_bal,loan_condition_int
42535,885.46,0.0,14.0,0.0,36638.0,61.2,21251.04,21251.04,4219.94,0.0,114834.0,1
42536,333.14,0.0,12.0,0.0,7967.0,52.8,7994.83,7994.83,1955.79,0.0,14123.0,1
42537,368.45,1.0,12.0,0.0,13168.0,21.6,8842.80,8842.80,1109.42,0.0,267646.0,1
42538,407.40,0.0,7.0,2.0,5572.0,68.8,13359.77,13359.77,1359.78,15386.0,13605.0,1
42539,476.30,0.0,8.0,0.0,11431.0,74.2,11431.20,11431.20,1880.47,1514.0,272492.0,1


In [13]:
Y = loan['int_rate']

In [17]:
# define base model
def baseline_model():
    
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [18]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

### 5 fold cross validation:

In [ ]:
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Instructions for updating:
Use tf.cast instead.


### Standardised Dataset

In [ ]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

### HyperParameter Tuning:

### Epochs:

In [ ]:
#Adding Layers:
# define the model
def larger_model():
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

### Optimizer:

In [ ]:
#Adding Layers:
# define the model
def larger_model():
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    from keras import optimizers
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [ ]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

### Learning Rate:

In [ ]:
#Adding Layers:
# define the model
def larger_model():
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
# Compile model
    from keras import optimizers
    sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [ ]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))